In [1]:
import pandas as pd
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import sample

In [2]:
nodes = pd.read_pickle("../../data/mock/mock_nodes.pkl")
channels = pd.read_pickle("../../data/mock/mock_channels.pkl")

In [3]:
nodes.to_csv("../../data/mock/mock_nodes.csv")
channels.to_csv("../../data/mock/mock_channels.csv")

In [4]:
## Note that the following are arbitrary policies, that can be changed as needed.
## Deal with multi-edges (aka multiple channels between two peers):
## - average rate fee
## - average base fee,
## - average capacity
## - keep one of the two channel ids
aggregation_dict = {
    "channel_id": "first",
    "rate_fee": "mean",
    "base_fee": "mean",
    "capacity": "sum"
}
channels.reset_index(inplace=True)
channels = channels.groupby(["node1_pub", "node2_pub"]).agg(aggregation_dict)
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

In [5]:
## To delete from [outgoing] and [incoming] lists the multi-edge
for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "outgoing_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "outgoing_channels"] = chan

for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "incoming_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "incoming_channels"] = chan

In [6]:
# Add to the nodes dataset the list of peers (nodes who have a channel with the N node)
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
nodes["peers"] = None
for n in nodes.index:
    peers = []
    for c in nodes.loc[n, "outgoing_channels"]:
        source, destination = channels[channels["channel_id"]==c].index[0]
        #print(f"For node {n}, the channel {c} is {source} --> {destination}")
        try:
            assert source == n
            peers.append(destination)
        except AssertionError as e:
            print(f"{source} is NOT equal to {n} for channel {c}")
    #print(peers)
    nodes.at[n, "peers"] = peers
channels.reset_index(inplace=True) 
channels.set_index("channel_id", inplace=True)

In [7]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [8]:

def create_demand(pd_object: pd.DataFrame) -> pd.DataFrame:
    """
    This function assigns the role of sender and receiver to
    two random nodes in the network
    :param pd_object: nodes dataframe
    :return: nodes dataset with demand column
    """
    #random.seed(874631)
    counterparties = sample(pd_object.index.to_list(), 2)
    sender = counterparties[0]
    receiver = counterparties[1]
    # Amounts in millisat (aka 10'000'000 is 10'000 sats)
    amount = 1524560 #random.randint(a=10000000, b=30000000)

    print(
        f"Transaction of {amount} sats from {pd_object[pd_object.index == sender]['alias'].item()} to {pd_object[pd_object.index == receiver]['alias'].item()}.")

    pd_object["demand"] = 0
    pd_object.loc[pd_object.index == sender, "demand"] = -amount
    pd_object.loc[pd_object.index == receiver, "demand"] = amount

    return pd_object


nodes = create_demand(nodes)



Transaction of 1524560 sats from UZHIMMBPBI.com to ISGMDIAZUV.com.


In [9]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [10]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [11]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS) + sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

In [12]:
## Single path constrain
def number_path_rule(model: pyo.ConcreteModel, n):
    outgoing = [model.x[(i, j)] for i, j in channels.index if i == n]
    incoming = [model.x[(i, j)] for i, j in channels.index if j == n]
    #outgoing = sum(model.x[(i, j)].value for j in nodes.at[n, "peers"] if i == n)
    #incoming = sum(model.x[(i, j)].value for i in nodes.at[n, "peers"] if j == n)
    return sum(incoming) == sum(outgoing)

model.NumberPathConstraint = pyo.Constraint(model.NODES, rule=number_path_rule)

In [13]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * model.x[(a, b)]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

In [14]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  OutFlow + nodes.loc[n, "demand"] == InFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)        

In [15]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())



Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpftt8j6kl.pyomo.lp -stat=1 -solve -solu /tmp/tmpftt8j6kl.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 883 (-3) rows, 1378 (-2) columns and 4078 (-10) elements
Statistics for presolved model
Original problem has 690 integers (690 of which binary)
Presolved problem has 689 integers (689 of which binary)
==== 77 zero objective 511 different
==== absolute objective values 511 different
==== for integers 1 zero objective 497 different
==== for integers absolute objective values 497 different
===== end objective counts


Problem has 883 rows, 1378 columns (1301 with objective) and 4078 elements
There are 2 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 689 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 689 

In [16]:
from decimal import Decimal
DF_channels = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    DF_channels.loc[c, "source"] = index[0]
    DF_channels.loc[c, "destination"] = index[1]
    DF_channels.loc[c, "capacity"] = channels.loc[index, "capacity"]
    DF_channels.loc[c, "amount"] = Decimal(value)
    c += 1
    
DF_channels_pos = DF_channels[DF_channels["amount"]!=0]
DF_channels_pos

,source,destination,capacity,amount
17,BTPMLOZRGV,MSOMRDUAUM,7523197.0,1524560
59,CPZVPUBAAH,TZTZCABFHU,6847789.0,1524560
276,MSOMRDUAUM,OQAPDLNBGO,7324301.0,1524560
320,OQAPDLNBGO,ISGMDIAZUV,6383098.0,1524560
380,PPNTQEQCAU,BTPMLOZRGV,7498283.0,1524560
532,TZTZCABFHU,PPNTQEQCAU,8823590.0,1524560
584,UZHIMMBPBI,CPZVPUBAAH,6387262.0,1524560


In [17]:
DF_channels[DF_channels["amount"]> DF_channels["capacity"]]

,source,destination,capacity,amount


In [18]:
#nodes.reset_index(inplace=True)
#DF_nodes = nodes[nodes["pub_key"].isin(DF_channels.loc[DF_channels["amount"] > 0, "source"].tolist()) | nodes["pub_key"].isin(DF_channels.loc[DF_channels["amount"] > 0, "destination"].tolist())]
#nodes.set_index("pub_key", inplace=True)
#
#DF_nodes

In [19]:
DF_fixed = pd.DataFrame()
c = 0
for index, value in model.x.extract_values().items():
    DF_fixed.loc[c, "source"] = index[0]
    DF_fixed.loc[c, "destination"] = index[1]
    DF_fixed.loc[c, "used"] = Decimal(value)
    c += 1
    
DF_fixed_pos = DF_fixed[DF_fixed["used"]!=0]

In [20]:
# Compare in order to catch not used channels that are selected by the fixed charge constrain
intersection = DF_fixed_pos.merge(DF_channels_pos, on=["source", "destination"], how="outer")
intersection

,source,destination,used,capacity,amount
0,BTPMLOZRGV,MSOMRDUAUM,1,7523197.0,1524560
1,BTPMLOZRGV,UZHIMMBPBI,1,NaN,NaN
2,CPZVPUBAAH,TZTZCABFHU,1,6847789.0,1524560
3,ISGMDIAZUV,OQAPDLNBGO,1,NaN,NaN
4,MSOMRDUAUM,OQAPDLNBGO,1,7324301.0,1524560
5,OQAPDLNBGO,ISGMDIAZUV,1,6383098.0,1524560
6,OQAPDLNBGO,QRFTLLBCSS,1,NaN,NaN
7,PGFZUBGVTF,BTPMLOZRGV,1,NaN,NaN
8,PPNTQEQCAU,BTPMLOZRGV,1,7498283.0,1524560
9,QRFTLLBCSS,RQTUFVOQNG,1,NaN,NaN
